# Edgar #

In this document we demonstrate how to use the Edgar Database for the selection of the stocks into a portfilio.

This functionality is using the https://pschatzmann.ch/edgar Webservice which is based on the data which is available on  https://www.sec.gov/edgar.shtml. 

### Setup  ###

We add the necessary jars and import the related packages.

In [31]:
%classpath config resolver maven-public https://software.pschatzmann.ch/repository/maven-public/
%classpath add mvn ch.pschatzmann:investor:1.1.0-SNAPSHOT
%classpath add mvn ch.pschatzmann:jupyter-jdk-extensions:0.0.1-SNAPSHOT

// our stock evaluation framwork
import ch.pschatzmann.dates._
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.input._
import ch.pschatzmann.stocks.cache._
import ch.pschatzmann.stocks.accounting._
import ch.pschatzmann.stocks.accounting.kpi._
import ch.pschatzmann.stocks.execution._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.execution.price._
import ch.pschatzmann.stocks.parameters._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.strategy.optimization._
import ch.pschatzmann.stocks.strategy.allocation._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.integration._
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._

// java
import java.util.stream.Collectors
import java.util._
import java.lang._
import java.util.function.Consumer
import scala.collection.JavaConverters

/// jupyter custom displayer
import ch.pschatzmann.display._

import ch.pschatzmann.dates._
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.input._
import ch.pschatzmann.stocks.cache._
import ch.pschatzmann.stocks.accounting._
import ch.pschatzmann.stocks.accounting.kpi._
import ch.pschatzmann.stocks.execution._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.execution.price._
import ch.pschatzmann.stocks.parameters._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.strategy.optimization._
import ch.pschatzmann.stocks.strategy.allocation._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.integration._
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._
import java.util.stream.Collectors
import java.ut...

In [32]:
//org.apache.log4j.PropertyConfigurator.configure("log4j.properties")

Displayers.setup()
Context.setDefaultStartDate(Context.date("2010-01-01"))

Context.isCacheActive()


true

### EdgarUniverse  ###

We can use the EdgarUniverse to select stocks which are available in the Edgar Database. 
The data is selected by specifying one or mulitple parameters for one year or for multiple years if we define a list of multiplication factors which define the weights for each year:  E.g (1.0, 1.0, 1.0) is selecting 3 years where the values are fully counted.

If no parmameter is specified we use the following list: "NetIncomeLoss","ProfitLoss","OperatingIncomeLoss".

Here are a couple of examples:

In [33]:
// top 10 companyies with the highest Net Income in the year 2017
new EdgarUniverse(2017).list(10)

[<head><title>301 Moved Permanently</title></head>, <body bgcolor="white">, <center><h1>301 Moved Permanently</h1></center>, <hr><center>nginx/1.10.3 (Ubuntu)</center>, </body>, </html>]

In [34]:
// top 10 companyies with the highest Net Income in the year 2015
new EdgarUniverse(2015, Arrays.asList("NetIncomeLoss")).list(10)

[<head><title>301 Moved Permanently</title></head>, <body bgcolor="white">, <center><h1>301 Moved Permanently</h1></center>, <hr><center>nginx/1.10.3 (Ubuntu)</center>, </body>, </html>]

In [35]:
// top 10 companyies with the highest R&D in the year 2017
new EdgarUniverse(2015, Arrays.asList("ResearchAndDevelopmentExpense")).list(10)

[<head><title>301 Moved Permanently</title></head>, <body bgcolor="white">, <center><h1>301 Moved Permanently</h1></center>, <hr><center>nginx/1.10.3 (Ubuntu)</center>, </body>, </html>]

In [36]:
// top 10 companyies with the highest Net Income in the year 2015-2017 
new EdgarUniverse(2015, Arrays.asList(0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true).list(10)

[<head><title>301 Moved Permanently</title></head>, <body bgcolor="white">, <center><h1>301 Moved Permanently</h1></center>, <hr><center>nginx/1.10.3 (Ubuntu)</center>, </body>, </html>]

In [37]:
// top 10 companyies with the highest Net Income Increases in the year 2014-2017 
var u = new EdgarUniverse(2015, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)
u.setCalculatePercentChange(true)
u.list(10)

[<head><title>301 Moved Permanently</title></head>, <body bgcolor="white">, <center><h1>301 Moved Permanently</h1></center>, <hr><center>nginx/1.10.3 (Ubuntu)</center>, </body>, </html>]

In [38]:
// top 10 companyies with the highest Net Income Increases in the year 2015-2017 
var u = new EdgarUniverse(2015, Arrays.asList(0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)
u.setCalculatePercentChange(true)
u.list(10)

[<head><title>301 Moved Permanently</title></head>, <body bgcolor="white">, <center><h1>301 Moved Permanently</h1></center>, <hr><center>nginx/1.10.3 (Ubuntu)</center>, </body>, </html>]

### Selection of valid Traded Stocks  ###


We might need to filter the provided values from Edgar in order to make sure that we get only records which are valid on a defined trading platform:

In [39]:
var validStocks = new ListUniverse(new NasdaqUniverse().list(), new NyseUniverse().list()).list()

validStocks.size()

warning:  there were two deprecation warnings; re-run with -deprecation for details

In [40]:
var edgarUniverse = new EdgarUniverse(2015, Arrays.asList("ResearchAndDevelopmentExpense"))

var edgarStocks = edgarUniverse.list(10, validStocks)

<console>: 154

We will collect the result (Absolute Return) in the following list:

In [41]:
var resultMap = new java.util.TreeMap[String,Double]()

### Selection of the Stocks with the Highest Profits  ###

For the past n years we determine the top n stocks of the previos year and we execute the evaluation of the selected stock with the best strategies. The portfiolio of selected stocks is updated every year.

We determine the Universe and the trading strategy selection for the prior year and then execute the strategy for the current period. 

If a stock has been deselected (from the latest universe) the system waits for the next sell signal to liquidate the stock.

In [42]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01");
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account)
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart())
    var edgarUniverse = new EdgarUniverse(year)
    var stocks = edgarUniverse.list(10, validStocks)
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, stocks, reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i))
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profits", account.getKPIValue(KPI.AbsoluteReturn))

kpi.getMap()

<console>: 178

In [43]:
// create chart for total values
var totalHistory = HistoricValues.create(account.getTotalValueHistory(), "TotalValue")
var cashHistory = HistoricValues.create(account.getCashHistoryForAllDates(), "Cash")
var actualHistory = HistoricValues.create(account.getActualValueHistory(), "ActualValue")
var historyTable = Table.create(totalHistory, cashHistory, actualHistory)

new SimpleTimePlot {
    data = historyTable.seq
    columns = Seq("TotalValue","Cash","ActualValue")
    showLegend = false
}

<console>: 153

### Immediate Liquidation  ###

With executor.setImmediateLiquidationOfDiscontinuedStocks(true) we can make sure that the stock is liquidated at the beginning of the period. Per default this setting is set to false.

In our example however this does not make any difference.

In [44]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01") 
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account)
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)
executor.setImmediateLiquidationOfDiscontinuedStocks(true)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart())
    var edgarUniverse = new EdgarUniverse(year)
    var stocks = edgarUniverse.list(10, validStocks)
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, stocks, reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i));
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profits (immediate liquidation)",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 166

### The Highest NetIncomeLoss for the last 4 years  ###

In the examples above we did the selection of stocks from edgar based on one year. In the following example we use 4
years with decreasing weights into the past.

In [45]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart());
    var edgarUniverse =  new EdgarUniverse(year, Arrays.asList(0.1, 0.3, 0.5, 1.0), Arrays.asList("NetIncomeLoss"), true)                                
    var stocks = edgarUniverse.list(10, validStocks)    
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, stocks, reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i))
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profits (last 4 years)",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 165

In [46]:
Displayers.display(account.getKPIValueByStockID(KPI.AbsoluteReturn))

<console>: 153

### The Highest NetIncomeLoss % Changes for the last 4 years  ###

In the previos examples we used the absolte parameter values to select the stock. In the following example we use
the calculated percent increase to do the selection the method

    u3.setCalculatePercentChange(true, 1000000)               
    
activates the calculation of percentages but only for values > 1000000. We are not interrested in fast growing low profit stocks! 



In [47]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account)
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart())
    var edgarUniverse =  new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)                                
    edgarUniverse.setCalculatePercentChange(true, 1000000)                             
    var stocks = edgarUniverse.list(10, validStocks)    
    // select only entries for which we have rates
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, stocks, reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i));
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest % Change Profits (last 4 years)",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 166

In [48]:
Displayers.display(account.getKPIValueByStockID(KPI.AbsoluteReturn))

<console>: 153

### The top R & D for the last 4 years  ###

We assume that companies with higher R&D will be more prifitable in the futrue. Thefore we run a selection based on the
ResearchAndDevelopmentExpense parameter.

In [49]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart())
    var edgarUniverse =  new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("ResearchAndDevelopmentExpense"), true)                                
    var stocks = edgarUniverse.list(10, validStocks)    
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, stocks, reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i));
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest R&D (last 4 years)",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 165

### Combining Multiple Edgar Universes  ###

Finally we verfiy the result if we combine multiple Edgar universe queries. Out of the 30 selected strategies we use the top 10

In [50]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account)
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart());
    var u1 = new EdgarUniverse(year, Arrays.asList(0.1, 0.3, 0.5, 1.0), Arrays.asList("NetIncomeLoss"), true)
    var u2 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("ResearchAndDevelopmentExpense"), true)
    var u3 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)                             
    u3.setCalculatePercentChange(true, 1000000)                             
    var listUniverse = new ListUniverse(u1.list(10,validStocks), u2.list(10,validStocks), u3.list(10,validStocks))                              
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, listUniverse.list(), reader)
    executor.setStrategies(result.getStrategies(reader));
    executor.run(periods.get(i));
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profit,Profit Growth, R&D (last 4 years) top 10",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 168

We combine multiple Edgar universe queries. Out of the 30 selected strategies we use all strategies

In [51]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account)
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart());
    var portfolioUniverse = new ListUniverse()
    var u1 = new EdgarUniverse(year, Arrays.asList(0.1, 0.3, 0.5, 1.0), Arrays.asList("NetIncomeLoss"), true)
    var u2 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("ResearchAndDevelopmentExpense"), true)
    var u3 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)                            
    u3.setCalculatePercentChange(true, 1000000)                             
    var listUniverse = new ListUniverse(u1.list(10, validStocks), u2.list(10, validStocks), u3.list(10, validStocks))
                              
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(30, listUniverse.list(), reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i))
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profit,Profit Growth, R&D (last 4 years) 30 stocks",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 169

### The Best of the Best ###

We anly take the stocks which have the best NetIncome AND R%D AND Income Growth:

In [52]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = TradingStrategyFactory.list()
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart());
    var u1 = new EdgarUniverse(year, Arrays.asList(0.1, 0.3, 0.5, 1.0), Arrays.asList("NetIncomeLoss"), true)
    var u2 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("ResearchAndDevelopmentExpense"), true)
    var u3 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)                            
    u3.setCalculatePercentChange(true, 1000000)   
    
    var listUniverse = new ListUniverse(u1.list(50, validStocks))
    listUniverse.retainAll(u2.list(50, validStocks))
    listUniverse.retainAll(u3.list(50, validStocks))
                              
    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, listUniverse.list(), reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i))
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profit & Profit Growth & R&D (last 4 years) top 10",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 169

### Evaluation by Stock  ###

Finally we check the contribution of each stock to the final result

In [53]:
account.getStockIDs()

<console>: 153

In [54]:
Displayers.display(account.getKPIValueByStockID(KPI.AbsoluteReturn))

<console>: 153

### Baseline: Combining Multiple Edgar Universes (Buy and Hold) ###

We just use the BuyAndHoldStrategy to evaluate the result

In [55]:
var reader = Context.getDefaultReader()
var periods = Context.getDateRanges("2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01","2016-01-01","2017-01-01")
var account = new Account("Simulation", "USD", 100000.00, periods.get(0).getStart(), new PerTradeFees(6.95))
var strategies = Arrays.asList("BuyAndHoldStrategy")
var trader = new PaperTrader(account);
var allocationStrategy = new DistributedAllocationStrategy(trader)
var executor = new StrategyExecutor(trader, allocationStrategy)

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart());
    var listUniverse = new ListUniverse()
    var u1 = new EdgarUniverse(year, Arrays.asList(0.1, 0.3, 0.5, 1.0), Arrays.asList("NetIncomeLoss"), true)
    var u2 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("ResearchAndDevelopmentExpense"), true)
    var u3 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)                            
    u3.setCalculatePercentChange(true, 1000000)                             
    listUniverse.add(u1.list(10, validStocks), u2.list(10, validStocks), u3.list(10, validStocks))

    var strategySelector = new StrategySelector(account, strategies, periods.get(i-1), KPI.AbsoluteReturn)
    var stockSelector = new StockSelector(strategySelector)
    var result = stockSelector.getSelection(10, listUniverse.list(), reader)
    executor.setStrategies(result.getStrategies(reader))
    executor.run(periods.get(i))
}  
account.setCloseDate(periods.get(5).getEnd())                                    
var kpi = account.getKPIValues()
resultMap.put("Highest Profit,Profit Growth, R&D -Buy and Hold- (last 4 years)",account.getKPIValue(KPI.AbsoluteReturn))

kpi

<console>: 169

### Final Comparision  ###


In [56]:
Displayers.display(resultMap)

Key,Value


In [57]:
resultMap

### Survivorship Bias ##

The result suffers from survivorship bias: We ignore the stocks that have not survived because we filter on the stocks which are traded today. So lets get the list of stocks which were ignored and double check if there companies went bust...

In [58]:
var listUniverse = new ListUniverse()

for (i <- 1 to 5) { 
    var year = Context.getYear(periods.get(i-1).getStart());
    var u1 = new EdgarUniverse(year, Arrays.asList(0.1, 0.3, 0.5, 1.0), Arrays.asList("NetIncomeLoss"), true)
    var u2 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("ResearchAndDevelopmentExpense"), true)
    var u3 = new EdgarUniverse(year, Arrays.asList(0.2, 0.5, 0.8, 1.0), Arrays.asList("NetIncomeLoss"), true)                            
    u3.setCalculatePercentChange(true, 1000000)                             
    listUniverse.add(u1.list(10), u2.list(10), u3.list(10))
}

listUniverse.removeAll(validStocks)
 
llistUniverse


<console>: 167

In [59]:

new IEXUniverse().list().contains(new StockID("GOOGL",""))

true

      
- BRKA
- EVEP: Chapter 11(-)
- GOOGL: THIS IS TRADED ON NASDAQ (OK because we selected GOOG)
- LBMH: Merger and Delisting (+)
- TORM: Delisting ()
- GORO: not on nyse and nasdaq (available in IEXUnivers) ()

So in a nutsehll the positive events (+) are outnumbering the negative events (-). 

